## DATA EXPLORATION ANALYSIS

### 1) KPI metrics 
    - DAU (daily active user)
    - MAU (month active user)
    - ARPU 

### 2) Analysis 
    - Trend
    - Issue of data 


In [1]:
import pandas as pd 
from sqlalchemy import create_engine
%matplotlib inline
%pylab inline
import seaborn  as sns 
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [3]:
# UDF 
def get_db_data(sql):
    try:
        engine = create_engine('sqlite:///data/war_data.db', echo=False)
        print (sql)
        df = pd.read_sql(sql, engine)
        return df
    except Exception as e:
        print ('query data failed', str(e))
    
def get_date(timestamp):
    return timestamp[:10]

def get_hour(timestamp):
    return timestamp[11:13]

In [4]:
# get data 
engine = create_engine('sqlite:///data/war_data.db', echo=False)
sql_session = "select * from session;"
df_session = pd.read_sql(sql_session, engine)
sql_battle = "select * from battle;"
df_battle = pd.read_sql(sql_battle, engine)
sql_cost = "select * from cost;"
df_cost = pd.read_sql(sql_cost, engine)

## 1) KPI metric

> 1-1) DAU : user that login/access/view/pay/play game on the APP per day 

In [5]:
# DAU : user that login/access/view/pay/play game on the APP per day 

sql_dau = """
SELECT substr(event_timestamp, 1, 10) AS date,
       count(DISTINCT user_id) AS user_count
FROM SESSION
GROUP BY 1
ORDER BY 1;

"""
df_dau = get_db_data(sql_dau)


SELECT substr(event_timestamp, 1, 10) AS date,
       count(DISTINCT user_id) AS user_count
FROM SESSION
GROUP BY 1
ORDER BY 1;




In [7]:
df_dau.T

,0,1,2,3,4,5,6,7,8,9,...,338,339,340,341,342,343,344,345,346,347
date,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,...,2018-12-05,2018-12-06,2018-12-07,2018-12-08,2018-12-09,2018-12-10,2018-12-11,2018-12-12,2018-12-13,2018-12-14
user_count,5,7,9,15,19,20,21,26,20,18,...,18,23,23,23,19,13,9,4,1,1


> 1-2) ARPU  : Average revenue per user 

In [9]:
# ARPU : Average revenue per user

sql_arpu = """

WITH dau AS
  (SELECT substr(event_timestamp, 1, 10) AS date,
          count(DISTINCT user_id) AS user_count
   FROM SESSION
   GROUP BY 1),
     revenue AS
  (SELECT substr(event_timestamp, 1, 10) AS date,
          sum(usd_cost) AS day_revenue
   FROM cost
   GROUP BY 1)
SELECT dau.date,
       revenue.day_revenue/dau.user_count AS arpu
FROM dau
LEFT JOIN revenue ON revenue.date = dau.date;

"""
df_arpu = get_db_data(sql_arpu)



WITH dau AS
  (SELECT substr(event_timestamp, 1, 10) AS date,
          count(DISTINCT user_id) AS user_count
   FROM SESSION
   GROUP BY 1),
     revenue AS
  (SELECT substr(event_timestamp, 1, 10) AS date,
          sum(usd_cost) AS day_revenue
   FROM cost
   GROUP BY 1)
SELECT dau.date,
       revenue.day_revenue/dau.user_count AS arpu
FROM dau
LEFT JOIN revenue ON revenue.date = dau.date;




In [10]:
df_arpu.T

,0,1,2,3,4,5,6,7,8,9,...,338,339,340,341,342,343,344,345,346,347
date,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,...,2018-12-05,2018-12-06,2018-12-07,2018-12-08,2018-12-09,2018-12-10,2018-12-11,2018-12-12,2018-12-13,2018-12-14
arpu,NaN,NaN,1.11,4.13333,NaN,NaN,0.190476,0.346154,NaN,0.776667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


> 1-3) ARPPU : Average revenue per pay user

In [11]:
df_session.head(3)

,index,client_version,event_name,event_timestamp,is_premium,login_attempt_id,platform,player_name,server_version,session_id,user_device_country,user_id,user_ip,user_is_spender,user_server_region,user_type
0,0,1.3.0,session_started,2018-03-01 07:10:48.499895,0,89c571bd-16de-4785-a051-f10aa27d54fa,IOS_MOBILE,luskfmpg,0.1.220,82e80eb5-42cd-4ed7-92c7-6e280f7bbcf1,US,85dff3d5-6f56-4601-9a24-400624f9a024,253.122.152.86,0,US,PLAYER
1,1,1.3.0,session_started,2018-05-01 13:58:47.838814,0,7ff39fa8-819c-4553-8cd0-4b000d338191,IOS_TABLET,myggq,0.1.220,0b76d245-a586-4b0c-a405-3dd09f255a65,US,0c91ed1d-40cc-419a-bd28-7a371e191f70,38.192.175.54,0,US,PLAYER
2,2,1.3.0,session_started,2018-09-29 22:42:47,0,d7c3b9fe-2464-45b2-90c8-825d3bb08335,IOS_TABLET,xuaushlwm,0.1.220,bfc766f4-ed5c-41a8-b887-f11378b3e483,US,d0c8f5e2-2f19-42dc-b279-0d7420ba90a2,89.252.254.157,0,EU,PLAYER


In [12]:
df_cost.head(3)

,index,client_version,event_name,event_timestamp,is_premium,platform,player_name,product_name,real_currency_amount,real_currency_type,server_version,usd_cost,user_device_country,user_id,user_ip,user_is_spender,user_server_region,user_type
0,0,1.3.0,in_app_purchase_log_server,2018-10-25 08:24:27.164771,0,IOS_MOBILE,woiogu,gems_medium,9.99,USD,0.1.220,9.99,US,387e9319-0322-4aa7-bd1b-849ec4549236,85.185.154.2,1,US,PLAYER
1,1,1.3.0,in_app_purchase_log_server,2018-09-06 13:53:43.766954,0,IOS_MOBILE,mfyqqump,gems_small,4.99,USD,0.1.220,4.99,US,104865e8-b05c-43d4-91e0-9cd47eb4e336,68.141.250.207,1,US,PLAYER
2,2,1.3.0,in_app_purchase_log_server,2018-04-26 19:40:38.635197,0,IOS_MOBILE,ksebgpggrxi,gems_large,"49,99",EUR,0.1.220,"49,99",US,ce4ec61d-f1ff-4ac6-bad4-fd402242b8f3,218.132.111.36,1,EU,PLAYER


In [13]:
# ARPPU : Average revenue per pay user

sql_arppu = """

WITH day_pay_user AS
  (SELECT substr(event_timestamp, 1, 10) AS date,
          count(DISTINCT user_id) AS user_count
   FROM cost
   GROUP BY 1),
     revenue AS
  (SELECT substr(event_timestamp, 1, 10) AS date,
          sum(usd_cost) AS day_revenue
   FROM cost
   GROUP BY 1)
SELECT day_pay_user.date,
       revenue.day_revenue/day_pay_user.user_count AS arppu
FROM day_pay_user
LEFT JOIN revenue ON revenue.date = day_pay_user.date;


"""
df_arppu = get_db_data(sql_arppu)



WITH day_pay_user AS
  (SELECT substr(event_timestamp, 1, 10) AS date,
          count(DISTINCT user_id) AS user_count
   FROM cost
   GROUP BY 1),
     revenue AS
  (SELECT substr(event_timestamp, 1, 10) AS date,
          sum(usd_cost) AS day_revenue
   FROM cost
   GROUP BY 1)
SELECT day_pay_user.date,
       revenue.day_revenue/day_pay_user.user_count AS arppu
FROM day_pay_user
LEFT JOIN revenue ON revenue.date = day_pay_user.date;





In [14]:
df_arpu.T

,0,1,2,3,4,5,6,7,8,9,...,338,339,340,341,342,343,344,345,346,347
date,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,...,2018-12-05,2018-12-06,2018-12-07,2018-12-08,2018-12-09,2018-12-10,2018-12-11,2018-12-12,2018-12-13,2018-12-14
arpu,NaN,NaN,1.11,4.13333,NaN,NaN,0.190476,0.346154,NaN,0.776667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2) Insight

> 2-1) Trend : battle 

In [15]:
df_battle.platform.value_counts()

IOS_TABLET        8482
ANDROID_MOBILE    8419
IOS_MOBILE        6986
Name: platform, dtype: int64

In [24]:
# df_battle_ = df_battle[df_battle.platform=='IOS_TABLET']
# df_IOS_TABLET_battle_count = df_battle_.groupby('hour').count()['battle_id'].reset_index()

# df_battle_ = df_battle[df_battle.platform=='ANDROID_MOBILE']
# df_ANDROID_MOBILE_battle_count = df_battle_.groupby('hour').count()['battle_id'].reset_index()

# df_battle_ = df_battle[df_battle.platform=='IOS_MOBILE']
# df_IOS_MOBILE_battle_count = df_battle_.groupby('hour').count()['battle_id'].reset_index()
  
    
# sns.lineplot(x="hour", 
#              y="battle_id",
#              data=df_IOS_TABLET_battle_count)

# sns.lineplot(x="hour", 
#              y="battle_id",
#              data=df_ANDROID_MOBILE_battle_count)

# sns.lineplot(x="hour", 
#              y="battle_id",
#              data=df_IOS_MOBILE_battle_count)
# plt.show()

> 2-2) Trend : event session

In [17]:
df_battle.event_name.value_counts()

multiplayer_battle_ended    23887
Name: event_name, dtype: int64

In [18]:
df_session.event_name.value_counts()

session_started    16375
Name: event_name, dtype: int64

In [19]:
df_cost.event_name.value_counts()

in_app_purchase_log_server    166
Name: event_name, dtype: int64

In [20]:
df_session.head(3)

,index,client_version,event_name,event_timestamp,is_premium,login_attempt_id,platform,player_name,server_version,session_id,user_device_country,user_id,user_ip,user_is_spender,user_server_region,user_type
0,0,1.3.0,session_started,2018-03-01 07:10:48.499895,0,89c571bd-16de-4785-a051-f10aa27d54fa,IOS_MOBILE,luskfmpg,0.1.220,82e80eb5-42cd-4ed7-92c7-6e280f7bbcf1,US,85dff3d5-6f56-4601-9a24-400624f9a024,253.122.152.86,0,US,PLAYER
1,1,1.3.0,session_started,2018-05-01 13:58:47.838814,0,7ff39fa8-819c-4553-8cd0-4b000d338191,IOS_TABLET,myggq,0.1.220,0b76d245-a586-4b0c-a405-3dd09f255a65,US,0c91ed1d-40cc-419a-bd28-7a371e191f70,38.192.175.54,0,US,PLAYER
2,2,1.3.0,session_started,2018-09-29 22:42:47,0,d7c3b9fe-2464-45b2-90c8-825d3bb08335,IOS_TABLET,xuaushlwm,0.1.220,bfc766f4-ed5c-41a8-b887-f11378b3e483,US,d0c8f5e2-2f19-42dc-b279-0d7420ba90a2,89.252.254.157,0,EU,PLAYER


In [21]:
# end of analysis